In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
from sklearn.linear_model import Ridge

In [5]:
train = pd.read_csv('../data/train.csv.bz2')
test = pd.read_csv('../data/test.csv.bz2')
submission = pd.read_csv('../data/sample_submission.csv.bz2')
#train = pd.read_csv('/kaggle/input/liverpool-ion-switching/train.csv')
#test = pd.read_csv('/kaggle/input/liverpool-ion-switching/test.csv')
#submission = pd.read_csv('/kaggle/input/liverpool-ion-switching/sample_submission.csv')

In [6]:
train.head()

,time,signal,open_channels
0,0.0001,-2.7600,0
1,0.0002,-2.8557,0
2,0.0003,-2.4074,0
3,0.0004,-3.1404,0
4,0.0005,-3.1525,0


In [7]:
train.tail()

,time,signal,open_channels
4999995,499.9996,2.9194,7
4999996,499.9997,2.6980,7
4999997,499.9998,4.5164,8
4999998,499.9999,5.6397,9
4999999,500.0000,5.3792,9


In [8]:
train.loc[49000:50010,:]

,time,signal,open_channels
49000,4.9001,-2.9378,0
49001,4.9002,-2.7240,0
49002,4.9003,-2.5879,0
49003,4.9004,-2.3940,0
49004,4.9005,-2.3708,0
49005,4.9006,-3.1857,0
49006,4.9007,-2.9674,0
49007,4.9008,-2.5549,0
49008,4.9009,-2.9567,0
49009,4.9010,-3.2541,0


In [9]:
train.shape

(5000000, 3)

In [10]:
train['open_channels'].min()

0

In [11]:
train_time = train['time'].values

In [12]:
train_time_0 = train_time[:50000]

In [13]:
train_time_0 = list(train_time_0) * 100

In [14]:
len(train_time_0)

5000000

In [15]:
train['time'] = train_time_0

In [17]:
test.head()

,time,signal
0,500.0001,-2.6498
1,500.0002,-2.8494
2,500.0003,-2.8600
3,500.0004,-2.4350
4,500.0005,-2.6155


In [18]:
test.tail()

,time,signal
1999995,699.9996,-2.9092
1999996,699.9997,-2.7422
1999997,699.9998,-2.8285
1999998,699.9999,-2.9092
1999999,700.0000,-2.7422


In [19]:
test.shape

(2000000, 2)

In [20]:
train_time_0 = train_time[:50000]
train_time_0 = list(train_time_0) * 40
test['time'] = train_time_0

In [21]:
n_groups = 100
train["group"] = 0
for i in range(n_groups):
    ids = np.arange(i * 50000, (i + 1) * 50000)
    train.loc[ids, "group"] = i
    
n_groups = 40
test["group"] = 0
for i in range(n_groups):
    ids = np.arange(i * 50000, (i + 1) * 50000)
    test.loc[ids, "group"] = i
    
train['signal_2'] = 0
test['signal_2'] = 0

n_groups = 100
for i in range(n_groups):
    sub = train[train.group == i]
    signals = sub.signal.values
    imax, imin = math.floor(np.max(signals)), math.ceil(np.min(signals))
    signals = (signals - np.min(signals)) / (np.max(signals) - np.min(signals))
    signals = signals * (imax - imin)
    train.loc[sub.index, "signal_2"] = [0,] + list(np.array(signals[:-1]))
    
n_groups = 40
for i in range(n_groups):
    sub = test[test.group == i]
    signals = sub.signal.values
    imax, imin = math.floor(np.max(signals)), math.ceil(np.min(signals))
    signals = (signals - np.min(signals)) / (np.max(signals) - np.min(signals))
    signals = signals * (imax - imin)
    test.loc[sub.index, "signal_2"] = [0,] + list(np.array(signals[:-1]))

In [22]:
X = train[['time', 'signal_2']].values
y = train['open_channels'].values

In [23]:
model = Ridge()
model.fit(X, y)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [24]:
train_preds = model.predict(X)

In [25]:
train_preds = np.clip(train_preds, 0, 10)

In [26]:
train_preds.mean()

2.7368485408846013

In [27]:
train_preds = train_preds.astype(int)

In [28]:
X_test = test[['time', 'signal_2']].values

In [29]:
submission.head()

,time,open_channels
0,500.0001,0
1,500.0002,0
2,500.0003,0
3,500.0004,0
4,500.0005,0


In [30]:
submission.shape

(2000000, 2)

In [31]:
X_test.shape

(2000000, 2)

In [32]:
test_preds = model.predict(X_test)
test_preds = np.clip(test_preds, 0, 10)
test_preds = test_preds.astype(int)
submission['open_channels'] = test_preds

In [33]:
test_preds.mean()

1.2352345

In [34]:
submission.head(20)

,time,open_channels
0,500.0001,0
1,500.0002,0
2,500.0003,0
3,500.0004,0
4,500.0005,0
5,500.0006,0
6,500.0007,0
7,500.0008,0
8,500.0009,0
9,500.0010,0


In [35]:
np.set_printoptions(precision=4)

In [36]:
submission.time.values[:20]

array([500.0001, 500.0002, 500.0003, 500.0004, 500.0005, 500.0006,
       500.0007, 500.0008, 500.0009, 500.001 , 500.0011, 500.0012,
       500.0013, 500.0014, 500.0015, 500.0016, 500.0017, 500.0018,
       500.0019, 500.002 ])

In [37]:
submission['time'] = [format(submission.time.values[x], '.4f') for x in range(2000000)]

In [38]:
submission.time.values[:20]

array(['500.0001', '500.0002', '500.0003', '500.0004', '500.0005',
       '500.0006', '500.0007', '500.0008', '500.0009', '500.0010',
       '500.0011', '500.0012', '500.0013', '500.0014', '500.0015',
       '500.0016', '500.0017', '500.0018', '500.0019', '500.0020'],
      dtype=object)

In [39]:
submission['open_channels'].mean()

1.2352345

In [40]:
submission.head()

,time,open_channels
0,500.0001,0
1,500.0002,0
2,500.0003,0
3,500.0004,0
4,500.0005,0


In [41]:
submission.to_csv('simple-ion-ridge-regression-starter-submission.csv', index=False)

In [ ]:
!gzip -9 simple-ion-ridge-regression-starter-submission.csv